In [ ]:
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt

# Projekt og dataset
PROJECT_ID = "pyne-dogs-ttm-cles"   # ret hvis dit projekt id er anderledes
DATASET = "bronze_prod"             # det du skrev

client = bigquery.Client(project=PROJECT_ID)

print("Client initialiseret:", client.project)


In [ ]:
sql_dim = f"""
SELECT
  breed_id,
  breed_name,
  temperament,
  breed_group,
  bred_for,
  is_family_friendly
FROM `{PROJECT_ID}.{DATASET}.dim_breed`
"""

sql_fact = f"""
SELECT
  breed_id,
  breed_name,
  weight_kg_min,
  weight_kg_max,
  weight_kg_avg,
  weight_class,
  lifespan_years_min,
  lifespan_years_max,
  lifespan_years_avg
FROM `{PROJECT_ID}.{DATASET}.fact_weight_life_span`
"""

dim_df = client.query(sql_dim).to_dataframe()
fact_df = client.query(sql_fact).to_dataframe()

dim_df.head(), fact_df.head()


In [ ]:
# Antag at breed_id er unik i dim_breed
df = fact_df.merge(
    dim_df[["breed_id", "temperament", "breed_group", "bred_for", "is_family_friendly"]],
    on="breed_id",
    how="left"
)

df.head()


In [ ]:
top10_lifespan = (
    df
    .dropna(subset=["lifespan_years_avg"])
    .sort_values("lifespan_years_avg", ascending=False)
    .head(10)
    [["breed_name", "lifespan_years_min", "lifespan_years_max", "lifespan_years_avg", "weight_class"]]
)

top10_lifespan


In [ ]:
top10_lifespan.to_html("top10_lifespan.html", index=False)
print("Gemte top10_lifespan.html")


In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(top10_lifespan["breed_name"], top10_lifespan["lifespan_years_avg"])
plt.xticks(rotation=45, ha="right")
plt.ylabel("År")
plt.title("Top 10 hunderacer efter gennemsnitlig levetid")
plt.tight_layout()
plt.savefig("top10_lifespan.png", dpi=150)
plt.show()


In [ ]:
weight_counts = (
    df
    .dropna(subset=["weight_class"])
    .groupby("weight_class")["breed_id"]
    .nunique()
    .reset_index(name="num_breeds")
    .sort_values("num_breeds", ascending=False)
)

weight_counts


In [ ]:
plt.figure(figsize=(6, 4))
plt.bar(weight_counts["weight_class"], weight_counts["num_breeds"])
plt.xlabel("Vægtklasse")
plt.ylabel("Antal racer")
plt.title("Fordeling af racer efter vægtklasse")
plt.tight_layout()
plt.savefig("weight_class_distribution.png", dpi=150)
plt.show()


In [ ]:
family_df = df[df["is_family_friendly"] == True].copy()

family_df[["breed_name", "temperament"]].head(20)


In [ ]:
# Hvor mange family-friendly breeds pr weight_class
family_weight_counts = (
    family_df
    .groupby("weight_class")["breed_id"]
    .nunique()
    .reset_index(name="num_family_friendly_breeds")
    .sort_values("num_family_friendly_breeds", ascending=False)
)

family_weight_counts

